In [461]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from lxml import html
from lxml import etree
import json

In [462]:
def obtem_soup(url):
    """Obtem o objeto Soup para o artigo
    Recebe:
            url :: str
    Retorna:
            soup :: bs4.BeautifulSoup
    """
    
    headers = {'User-Agent': 
               'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    html_texto = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html_texto, 'html.parser')
    
    return soup

In [463]:
def processa_seletor(sel_bruto):
    """Processa a string bruta do seletor de CSS obtida no site para
    compatibilidade com a sitaxe esperada pelo Python/BeautifullSoup"""
    
    sel_processado = sel_bruto.replace('nth-child', 'nth-of-type')
    return sel_processado

In [464]:
def valor_int(str_valor):
    """Converte o valor da bolsa de string para float.
    Recebe:
            str_valor_usd :: str
    Retorna:
            valor_int :: int
    """
    chars_a_remover = ['$', 'U', 'S', ',', ' ']
    valor_inteiro = ''.join([char for char in str_valor if char not in chars_a_remover])
    return valor_inteiro

In [465]:
def remove_novas_linhas(str_bruta, sep='|'):
    """Remove novas linhas em excesso e 
    devolve uma string com separador=sep.
    Recebe:
            str_bruta :: str
    Retorna:
            str_proc  :: str
            
    Exemplo:
        str_bruta = '\n\nCampo 1:\n\n\nValor 1\n\nValor2\n\n\n'
        str_proc = '|Campo 1:|Valor 1|Valor2|'
    """
    return re.sub('\n+', sep, str_bruta).strip()

In [466]:
def parse_cabecalho(soup, sel_raiz):
    """Parse do cabecalho, Titulo e Investigators
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            Titulo :: str
            Investigator :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(1) > div > div > div > div > div'
    sel_css_titulo = processa_seletor(f'{sel_raiz} > {sel_tronco} > h1')
    sel_css_instituicao = processa_seletor(f'{sel_raiz} > {sel_tronco} > div > p')
    
    # Obtem o titulo e o investigador
    titulo = soup.select(sel_css_titulo)[0].text
    
    investigador_bruto = soup.select(sel_css_instituicao)[0].text
    investigador = investigador_bruto.split(':')[1].strip()
    
    return {'titulo': titulo, 'instituicao': investigador}

In [474]:
def parse_data_valor(soup, sel_raiz):
    """Parse do Data de Inicio, Valor da bolsa
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            Data :: str
            Valor :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(2)'
    sel_css_data = processa_seletor(f'{sel_raiz} > {sel_tronco} > div:nth-child(1)')
    sel_css_valor = processa_seletor(f'{sel_raiz} > {sel_tronco} > div:nth-child(2)')
    
    # Obtem a data inicial e o valor
    data_ini_bruta = soup.select(sel_css_data)[0].text
    data_ini = remove_novas_linhas(data_ini_bruta, sep='|').split('|')[2]
    
    valor_bruto = soup.select(sel_css_valor)[0].text
    valor_str  = remove_novas_linhas(valor_bruto, sep='|').split('|')[2]
    valor_num = valor_int(valor_str)
    
    return {'data_ini': data_ini, 'valor_bolsa': valor_num}

In [468]:
def parse_descricao_detalhada(soup, sel_raiz):
    """Parse da descricao detalhada 
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            descricao_detalhada :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(3) > div > div > div > div > div > div'
    sel_css_descr_det = processa_seletor(f'{sel_raiz} > {sel_tronco}')
    
    # Obtem a data inicial e o valor
    descricao_detalhada_bruta = soup.select(sel_css_descr_det)[0].text
    descricao_detalhada = remove_novas_linhas(descricao_detalhada_bruta, sep='\n')
  
    return {'descr_detalhada': descricao_detalhada}

In [475]:
def parse_artigo(soup, sel_raiz_artigo):
    """Parse completo do artigo 
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            json contendo as seguintes informacoes:
                titulo :: str
                instituicao :: str
                data_ini :: str
                valor_bolsa :: int (USD)
                descricao_detalhada :: str
    """
    campos = {}
    
    # Obtem titulo e instituicao
    campos.update(parse_cabecalho(soup, sel_css_artigo))
    
    # Obtem data_ini e valor
    campos.update(parse_data_valor(soup, sel_css_artigo))
    
    # Obtem descr_detalhada
    campos.update(parse_descricao_detalhada(soup, sel_css_artigo))
    
    json_campos = json.loads(json.dumps(campos))
    return json_campos

In [476]:
url_base = 'https://erefdn.org/research-grants-projects/currently-funded-projects'
url_artigo = 'https://erefdn.org/non-recyclable-plastics-to-pavements/'

In [477]:
sel_css_artigo = '#post-14369'

In [480]:
soup = obtem_soup(url_artigo)
json_artigo = parse_artigo(soup, sel_css_artigo)
print(json.dumps(json_artigo, indent=4))

{
    "titulo": "Non-Recyclable Plastics to Pavements",
    "instituicao": "University of Illinois Urbana-Champaign",
    "data_ini": "TBD",
    "valor_bolsa": "161075",
    "descr_detalhada": "This proposal seeks to create high-value and high-volume products from plastic waste for bitumen (asphalt binder) replacement in pavements. The bitumen replacement market is a potential repurposing for large quantities of waste plastics. It addresses an urgent economic and environmental need for plastic recycling as well as the transportation industry. With 4-5% replacement of bitumen, this market has the potential to consume 1 million tons of waste plastics out of the 26 million tons that go to landfills in the US. Also, the study goal is aligned with the global emphasis on enhancing transportation infrastructure sustainability. Moreover, asphalt pavements are 100% recyclable; therefore, plastic waste will remain in a recycling circular loop. Plastic waste, from landfill destined municipal soli

In [473]:
class Bolsa():
    def __init__(self, url):
        self.titulo = ''
        self.data_inicio = ''
        self.valor_bolsa = ''
        self.site_origem = url
        self.descicao_detalhada = ''
        
    def obtem_dados(self):
        pass
        